In [3]:
from sklearn.decomposition import KernelPCA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import cross_validate
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import image
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold,SelectKBest,mutual_info_classif
from neural_tangents import stax

ModuleNotFoundError: jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.

In [ ]:
data = pd.read_csv('mnist_test.csv',header = None).iloc[5000:,:].sample(n=4000)
data_X = data.iloc[:,1:].to_numpy()
data_y = data.iloc[:,0].to_numpy()
data_test = pd.read_csv('mnist_test.csv',header = None).iloc[:5000,:].sample(n=4000)
data_test_X = data_test.iloc[:,1:].to_numpy()
data_test_y = data_test.iloc[:,0].to_numpy()

In [ ]:
#input: the data matrix
#return: NTK kernel matrix
def NTK(X,Y):
    # specify the NN to compute NTK
    init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Dense(512), stax.Relu(),
    stax.Dense(512), stax.Relu(),
    stax.Dense(1)
    )
    # compute kernel matrix
    ntk = kernel_fn(X, Y, 'ntk')
    print(ntk.shape)
    return k_matrix

In [ ]:
# parameter X: data array of shape(n_sample,n_feature)
# parameter y: labels of shape(n_samples,)
# parameter kernel: kernerl used for each layer (here is fixed for arccosine)
# parameter classifier: classifier used for the output layer
# parameter arc_order: the order of activation function and arccosine kernel
# return the trained MKM model

# classified by KNN with cv

def MKM(X,y,l,kernel,classifier,arc_order,layer = 1):
    New_X,New_y = X,y
    print(New_X.shape)
    train_list = []
    KPCA_list = []
    FS_list = []
    w_list = []
    for i in range(l):
        size = New_X.shape[1]
        # K-layer
        train_list.append(New_X)
        gram_train = ArcCosineKernel(New_X,New_X,arc_order)
        KPCA = KernelPCA(n_components = size-80, kernel = kernel)
        K_new_X,K_new_y = KPCA.fit_transform(gram_train),New_y
        # Bookkeeping the KPCA model used in each layer
        KPCA_list.append(KPCA)
        # P-layer
#         w = cv_w(K_new_X,K_new_y,range(1,4),range(10,30,5))
#         w_list.append(w)
#         P_new_X,P_new_y,dic_kept = mutual_info_FS(K_new_X,K_new_y,size-w)
#         P_new_X,P_new_y,dic_kept = mutual_info_FS(K_new_X,K_new_y,size-100)
#         P_new_X,P_new_y,dic_kept = mutual_info_FS(K_new_X,K_new_y,200)
#         FS_list.append(dic_kept)
#         New_X,New_y = P_new_X,P_new_y
        New_X,New_y = K_new_X,K_new_y
        print('Train Layer {} Done!'.format(i+1))
        print('Data has the shape {}'.format(New_X.shape))
    final_X, final_y =  New_X, New_y
    model = classifier.fit(final_X, final_y)
    print('Model has been successfully trained!')
    print('----------------------------------------------------------------------')
    return model,KPCA_list,FS_list,w_list,train_list

In [ ]:
# parameter X: data array of shape(n_sample,n_feature)
# parameter y: labels of shape(n_samples,)
# parameter data_test_X: data array of shape(n_sample,n_feature)
# parameter data_test_y: labels of shape(n_samples,)
# parameter l: the number of layers of kernel machines
# parameter MKM: the trained MKM model
# parameter kernel: kernerl used for each layer (here is fixed for arccosine)
# parameter arc_order: the order of activation function and arccosine kernel
# return accuracy score for test set

# classified by KNN with cv

def MKM__test(X,y,l,arc_order,KPCA_list,FS_list,w_list,train_list,MKM):
    New_X,New_y = X,y
    for i in range(l):
        size = New_X.shape[1]
        dic_feature = {}
        for f in range(size):
            dic_feature[f] = New_X[:,f]
        # K-layer
        gram_test = ArcCosineKernel(New_X,train_list[i],arc_order)
        K_new_X,K_new_y = KPCA_list[i].transform(gram_test),New_y
        # P-layer
#         w = cv_w(K_new_X,K_new_y,range(1,15),range(10,300))
#         P_new_X,P_new_y = mutual_info_FS(K_new_X,K_new_y,w)
#         P_new_X,P_new_y = mutual_info_FS(K_new_X,K_new_y,209)
#         for key in FS_list[i].keys():
#             FS_list[i][key] = dic_feature[key]
#         temp = np.array(list(FS_list[i].values())).T[:,:size-80-w_list[i]]
#         temp = np.array(list(FS_list[i].values())).T[:,:size-100]
#         P_new_X,P_new_y = temp,K_new_y
#         New_X,New_y = P_new_X,P_new_y
        New_X,New_y = K_new_X,K_new_y
        print('Test Layer {} Done!'.format(i+1))
        print('Data has the shape {}'.format(New_X.shape))
    final_X, final_y =  New_X, New_y
    predict_y = MKM.predict(final_X)
    print(accuracy_score(final_y,predict_y))
    print('Test is finished!')
    print('----------------------------------------------------------------------')
    return accuracy_score(final_y,predict_y)

In [ ]:
model, KPCA_list ,FS_list,w_list,train_list= MKM(data_X,data_y,1,'precomputed',KNN(10),0)

In [ ]:
MKM__test(data_test_X,data_test_y,1,0,KPCA_list,FS_list,w_list,train_list,model)